# Exploratory Data Analysis


### Definiera problemet och skapa en helhetsbild
- GOOD

### Få tillgång till datan
- GOOD

### Utforska datan, gör en exploratory data analysis (EDA)
- DONE (Fixa en snygg eda senare)

### Bearbeta datan
- TODO now

### ML-modellering
- Todo

### Presentera din lösning för intressenter
- Todo

### Produktionssättning av modellen och övervakning av implementeringen
- Todo


### Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.metrics import roc_auc_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance

from sklearn.impute import SimpleImputer

# 1.1 Ladda data
df = pd.read_csv("stackoverflow_full.csv")      # <- bytest dit filnamn om nödvändigt
df.head()


,Unnamed: 0,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,Employed
0,0,<35,No,Master,1,Man,No,Dev,7,4,Sweden,51552.0,C++;Python;Git;PostgreSQL,4,0
1,1,<35,No,Undergraduate,1,Man,No,Dev,12,5,Spain,46482.0,Bash/Shell;HTML/CSS;JavaScript;Node.js;SQL;Typ...,12,1
2,2,<35,No,Master,1,Man,No,Dev,15,6,Germany,77290.0,C;C++;Java;Perl;Ruby;Git;Ruby on Rails,7,0
3,3,<35,No,Undergraduate,1,Man,No,Dev,9,6,Canada,46135.0,Bash/Shell;HTML/CSS;JavaScript;PHP;Ruby;SQL;Gi...,13,0
4,4,>35,No,PhD,0,Man,No,NotDev,40,30,Singapore,160932.0,C++;Python,2,0


In [47]:
# 1.2 Grundläggande statistik
df.info()
df.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73462 entries, 0 to 73461
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      73462 non-null  int64  
 1   Age             73462 non-null  object 
 2   Accessibility   73462 non-null  object 
 3   EdLevel         73462 non-null  object 
 4   Employment      73462 non-null  int64  
 5   Gender          73462 non-null  object 
 6   MentalHealth    73462 non-null  object 
 7   MainBranch      73462 non-null  object 
 8   YearsCode       73462 non-null  int64  
 9   YearsCodePro    73462 non-null  int64  
 10  Country         73462 non-null  object 
 11  PreviousSalary  73462 non-null  float64
 12  HaveWorkedWith  73399 non-null  object 
 13  ComputerSkills  73462 non-null  int64  
 14  Employed        73462 non-null  int64  
dtypes: float64(1), int64(6), object(8)
memory usage: 8.4+ MB


,Unnamed: 0,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,Employed
count,73462.000000,73462,73462,73462,73462.000000,73462,73462,73462,73462.000000,73462.000000,73462,73462.000000,73399,73462.000000,73462.000000
unique,NaN,2,2,5,NaN,3,2,2,NaN,NaN,172,NaN,69980,NaN,NaN
top,NaN,<35,No,Undergraduate,NaN,Man,No,Dev,NaN,NaN,United States of America,NaN,Python,NaN,NaN
freq,NaN,47819,71355,37402,NaN,68573,56944,67396,NaN,NaN,14696,NaN,71,NaN,NaN
mean,36730.500000,NaN,NaN,NaN,0.883096,NaN,NaN,NaN,14.218902,9.098377,NaN,67750.260611,NaN,13.428221,0.536223
std,21206.797075,NaN,NaN,NaN,0.321308,NaN,NaN,NaN,9.405172,7.960201,NaN,49488.142118,NaN,7.057835,0.498690
min,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,1.000000,NaN,0.000000,0.000000
25%,18365.250000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,7.000000,3.000000,NaN,28839.000000,NaN,8.000000,0.000000
50%,36730.500000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,12.000000,7.000000,NaN,57588.000000,NaN,13.000000,1.000000
75%,55095.750000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,20.000000,12.000000,NaN,95979.000000,NaN,17.000000,1.000000


In [38]:
# 2.1 Konvertera ålder till numeriskt (≈30 och 40 för att kunna korrelera)
df['Age_num'] = df['Age'].map({'<35':34, '>35':36, '35-40':38, '>40':45})  # justera efter behov
df['Age_num'] = df['Age_num'].astype(int)

# 2.2 Ersätt oönskade kategorier om det finns
# 2.3 Skapa en dummy‑kolumn för `Employed` (0/1)
df['Employed'] = df['Employed'].astype(int)


In [39]:
# 3.3 Accessibility
print(df.groupby('Accessibility')['Employed'].mean())

# 3.5 MainBranch
print(df['MainBranch'].value_counts(normalize=True))

# 3.6 Country balans
country_balance = df['Country'].value_counts(normalize=True) * 100
print(country_balance)


Accessibility
No     0.535309
Yes    0.567157
Name: Employed, dtype: float64
MainBranch
Dev       0.917427
NotDev    0.082573
Name: proportion, dtype: float64
Country
United States of America                                20.004900
Germany                                                  7.343933
India                                                    7.296289
United Kingdom of Great Britain and Northern Ireland     6.381531
Canada                                                   3.782908
                                                          ...    
Mauritania                                               0.001361
Burundi                                                  0.001361
Saint Kitts and Nevis                                    0.001361
Monaco                                                   0.001361
Seychelles                                               0.001361
Name: proportion, Length: 172, dtype: float64


In [40]:
# 4.1 Rader med exakt samma värden (förutom Employed)
dup_rows = df.drop(columns=['Employed']).duplicated(keep=False)
print(f"Antal rader med likadana värden (excl. Employed): {dup_rows.sum()}")
df[dup_rows]

# 4.2 Negativa löner eller 0 år erfarenhet men hög lön
num_negative_salaries = len(df[df['PreviousSalary'] < 0])
print(f"Antal rader med negativ lön: {num_negative_salaries}")
print(df[(df['YearsCode'] == 0) & (df['PreviousSalary'] > 0)])

# 4.3 40‑års upplevelse men under 35 år gammal
print(df[(df['YearsCode'] > 40) & (df['Age_num'] < 35)])


Antal rader med likadana värden (excl. Employed): 0
Antal rader med negativ lön: 0
       Unnamed: 0  Age Accessibility        EdLevel  Employment Gender  \
1714         1714  <35            No  Undergraduate           1    Man   
5135         5135  <35            No  Undergraduate           1    Man   
5244         5244  <35            No  Undergraduate           1    Man   
5413         5413  <35            No          Other           0    Man   
5532         5532  <35            No  Undergraduate           1    Man   
...           ...  ...           ...            ...         ...    ...   
72649       72649  <35            No          Other           0    Man   
72997       72997  <35            No  Undergraduate           1    Man   
73164       73164  >35            No  Undergraduate           1  Woman   
73216       73216  >35            No     NoHigherEd           1    Man   
73340       73340  <35            No  Undergraduate           1    Man   

      MentalHealth MainBranc

In [ ]:


# --- Steg 1: Skapa Age_num ---
def get_age_num(age_str):
    if isinstance(age_str, str):
        if '-' in age_str:
            return int(age_str.split('-')[0])
        elif '<' in age_str:
            return int(age_str.strip('<')) - 5   # tex "<35" → 30
        elif '>' in age_str:
            return int(age_str.strip('>')) + 5   # tex ">35" → 40
    return np.nan

df['Age_num'] = df['Age'].apply(get_age_num)

# --- Steg 2: Definiera Features och Målvariabel ---
categorical_cols_onehot = ['Gender', 'Country', 'MainBranch']
categorical_cols_ordinal = ['EdLevel']
numeric_cols = ['YearsCode', 'YearsCodePro', 'ComputerSkills', 'PreviousSalary', 'Age_num']

edlevel_order = ['NoHigherEd', 'High school', 'Undergraduate', 'Master', 'PhD', 'Other']

# --- Steg 3: Preprocessor ---
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),   # fyll NaN i numeriska
])

categorical_transformer_ord = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(categories=[edlevel_order]))
])

categorical_transformer_oh = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('ord', categorical_transformer_ord, categorical_cols_ordinal),
        ('onehot', categorical_transformer_oh, categorical_cols_onehot),
        ('num', numeric_transformer, numeric_cols)
    ]
)

# --- Steg 4: Pipeline ---
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=200, random_state=42))
])

X = df[categorical_cols_ordinal + categorical_cols_onehot + numeric_cols]
y = df['Employed']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Träna modellen
rf_pipeline.fit(X_train, y_train)

# --- Steg 5: Permutation importance ---
perm = permutation_importance(
    rf_pipeline, X_test, y_test, n_repeats=30, random_state=42
)

# Hämta feature-namn från hela pipelinen
feature_names = rf_pipeline.named_steps['preprocessor'].get_feature_names_out(
    input_features=X.columns
)

# Nu matchar antalet features antalet permutation importances
importance = pd.Series(
    perm.importances_mean,
    index=feature_names
).sort_values(ascending=False)

print("Permutation Importance:")
print(importance.head(20))  # t.ex. visa topp 20


print("Permutation Importance:")
print(importance)


In [ ]:
# 6.1 Kvinnors andel som får anställning
women_employed = df[df['Gender']=='Woman']['Employed'].mean()
men_employed = df[df['Gender']=='Man']['Employed'].mean()
print(f"Kvinnor: {women_employed:.2%}, Män: {men_employed:.2%}")


In [ ]:
# Linjärt samband
sns.regplot(x='YearsCode', y='Employed', data=df)
plt.title('YearsCode vs Employed (linjär)')
plt.show()

# Små dataset → "sweet spot"-analys
df['YearsCode_bin'] = pd.cut(df['YearsCode'], bins=[0,5,10,20,40, np.inf],
                             labels=['<5','5-10','10-20','20-40', '40+'])
print(df.groupby('YearsCode_bin')['Employed'].mean())


In [ ]:
df['CodeProportion'] = df['YearsCodePro'] / df['YearsCode'].replace(0, np.nan)
df['CodeProportion'].hist(bins=30)
plt.title('CodeProportion fördelning')
plt.show()


In [ ]:
sns.boxplot(x='EdLevel', y='YearsCode', data=df, palette='Set2')
plt.title('YearsCode per utbildningsnivå')
plt.show()


In [ ]:
df['Age_YearsCode'] = df['Age_num'] * df['YearsCode']
sns.scatterplot(x='Age_YearsCode', y='Employed', data=df)
plt.title('Age × YearsCode vs Employed')
plt.show()


In [ ]:
# 8.1 Dummy‑kodning (för att hålla dimensionerna hanterbara)
feat_df = pd.get_dummies(df[['YearsCode', 'YearsCodePro', 'Age_num', 'Gender', 'Country', 'EdLevel_enc']], 
                         drop_first=True)

# 8.2 VIF
vif_data = pd.DataFrame()
vif_data["feature"] = feat_df.columns
vif_data["VIF"] = [variance_inflation_factor(feat_df.values, i)
                   for i in range(feat_df.shape[1])]
print(vif_data.sort_values(by="VIF", ascending=False).head(10))


In [ ]:
df['Salary_per_Exp'] = df['PreviousSalary'] / df['YearsCodePro'].replace(0, np.nan)
sns.lineplot(x='YearsCodePro', y='Salary_per_Exp', data=df)
plt.title('PreviousSalary / YearsCodePro')
plt.show()


In [ ]:
exchange_rates = {
    'Sweden': 10.3,
    'Spain': 1.1,
    'Germany': 1.1,
    'Canada': 1.25,
    # … lägg till alla länder som finns
}

df['Salary_USD'] = df.apply(lambda r: r['PreviousSalary'] / exchange_rates.get(r['Country'], 1), axis=1)


In [ ]:
X = df.drop(columns=['Employed','Age','Accessibility','MainBranch','HaveWorkedWith','ComputerSkills'])
y = df['Employed']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=df[['Country', 'Gender']]   # Stratifiserad efter land + kön
)


In [ ]:
# 13.1 Korrelation heatmap
plt.figure(figsize=(12,10))
sns.heatmap(df.corr(), annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Korrelation (numeriska kolumner)')
plt.show()

# 13.2 Sankey för huvudbransch
import plotly.graph_objects as go
main_df = df['MainBranch'].value_counts().reset_index()
main_df.columns = ['MainBranch','Count']
fig = go.Figure(data=[go.Pie(labels=main_df['MainBranch'], values=main_df['Count'], hole=.3)])
fig.update_layout(title_text='Fördelning av huvudbranscher')
fig.show()
